[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.leafmap.org/lab/index.html?path=maplibre/3d_choropleth.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/leafmap/blob/master/docs/maplibre/3d_choropleth.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeos/leafmap/HEAD)

**Create a 3D choropleth map of Europe with countries extruded**

This source code of this example is adapted from the MapTiler SDK JS example - [Create a 3D choropleth map of Europe with countries extruded](https://docs.maptiler.com/sdk-js/examples/fill-extrusion).

Uncomment the following line to install [leafmap](https://leafmap.org) if needed.

In [1]:
# %pip install "leafmap[maplibre]"

import os
import urllib.request, json 
import pandas as pd

In [2]:
import leafmap.maplibregl as leafmap

To run this notebook, you will need an [API key](https://docs.maptiler.com/cloud/api/authentication-key/) from [MapTiler](https://www.maptiler.com/cloud/). Once you have the API key, you can uncomment the following code block and replace `YOUR_API_KEY` with your actual API key. Then, run the code block code to set the API key as an environment variable.

In [3]:
# Load income data
INCOME_URL = "https://www.irs.gov/pub/irs-soi/21incyallagi.csv"
PD_INCOME = pd.read_csv(INCOME_URL, encoding='ISO-8859-1')

# Group by both STATEFIPS and COUNTYFIPS, and aggregate the N1 column
GROUPED = (
    PD_INCOME.groupby(["STATEFIPS", "COUNTYFIPS"], as_index=False)
    .agg(total_taxpayers=('N1', 'sum'))  # Sum N1 for each combination of STATEFIPS and COUNTYFIPS
)

print(GROUPED)

# Convert the grouped DataFrame to a dictionary with a tuple (STATEFIPS, COUNTYFIPS) as keys
GROUPED_DICT = dict(zip(zip(GROUPED['STATEFIPS'], GROUPED['COUNTYFIPS']), GROUPED['total_taxpayers']))

print(GROUPED_DICT)

# Calculate relative_percent using the composite key
PD_INCOME['relative_percent'] = PD_INCOME.apply(
    lambda row: 100 * row['N1'] / GROUPED_DICT.get((row["STATEFIPS"], row['COUNTYFIPS']), 1),
    axis=1
)
PD_INCOME['STATEFIPS'] = PD_INCOME['STATEFIPS'].astype(int).astype(str).str.zfill(2)  # Ensure two digits
PD_INCOME['COUNTYFIPS'] = PD_INCOME['COUNTYFIPS'].astype(int).astype(str).str.zfill(3)  # Ensure five digits
PD_INCOME['id'] = PD_INCOME['STATEFIPS'].astype(str) + PD_INCOME['COUNTYFIPS'].astype(str)



print(PD_INCOME.loc[20:30])  # Adjusted to show rows 20 to 30


      STATEFIPS  COUNTYFIPS  total_taxpayers
0             1           0        2162210.0
1             1           1          25720.0
2             1           3         111140.0
3             1           5           9740.0
4             1           7           8230.0
...         ...         ...              ...
3189         56          37          19530.0
3190         56          39          14860.0
3191         56          41           9350.0
3192         56          43           3790.0
3193         56          45           3190.0

[3194 rows x 3 columns]
{(1, 0): 2162210.0, (1, 1): 25720.0, (1, 3): 111140.0, (1, 5): 9740.0, (1, 7): 8230.0, (1, 9): 24110.0, (1, 11): 4100.0, (1, 13): 8100.0, (1, 15): 49290.0, (1, 17): 14200.0, (1, 19): 10450.0, (1, 21): 18670.0, (1, 23): 5000.0, (1, 25): 10130.0, (1, 27): 5750.0, (1, 29): 6180.0, (1, 31): 22960.0, (1, 33): 24820.0, (1, 35): 4860.0, (1, 37): 4020.0, (1, 39): 15490.0, (1, 41): 5810.0, (1, 43): 36900.0, (1, 45): 20830.0, (1, 47): 14870.

In [4]:
COUNTIES_URL = "https://open.gishub.org/data/us/us_counties.geojson"
with urllib.request.urlopen(COUNTIES_URL) as response:
    COUNTIES_JSON = json.load(response)
    PD_COUNTIES = pd.json_normalize(COUNTIES_JSON['features'])

# Iterate over the features in the GeoJSON
for feature in COUNTIES_JSON['features']:
    properties = feature.get('properties', {})
    state = properties.get('STATE')
    county = properties.get('COUNTY')
    
    if state is not None and county is not None:
        id = str(state) + str(county)  # Ensure both are strings
        if id:
            matching_rows = PD_INCOME[PD_INCOME['id'] == id]
            for _, row in matching_rows.iterrows():  # Use iterrows() to iterate over DataFrame rows
                properties[str(row['agi_stub']) + "total"] = row["N1"]  # Assign values to properties
                properties[str(row['agi_stub']) + "relative"] = row["relative_percent"]
print(COUNTIES_JSON)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [6]:
# Add GeoJSON source
source = {
    "type": "geojson",
    "data": COUNTIES_JSON,
}

# Create a map
m = leafmap.Map(center=[-100, 40], zoom=3, pitch=60, style="simple")


m.add_source('counties', source)

# Define the fill-extrusion layer
layer = {
    "id": "us-counties",
    "source": "counties",
    "type": "fill-extrusion",
    "paint": {
        "fill-extrusion-color": [
            "interpolate",
            ["linear"],
            ["get", "8relative"],
            0, "#ffffff",
            25, "#fee6ce",
            30, "#fdd0a2",
            50, "#ff0000",
        ],
        "fill-extrusion-opacity": 1,
        "fill-extrusion-height": ["*", ["get", "8total"], 1],
    },
}

# Add the layer to the map
m.add_layer(layer)

# # Define the names layer
# names_layer = {
#     "id": "us-counties-name",
#     "source": "counties",
#     "type": "symbol",
#     "layout": {
#         "text-field": ["get", "NAME"],  # Assuming 'NAME' is the field for county names
#         'text-variable-anchor': ['top', 'bottom', 'left', 'right'],
#         'text-radial-offset': 0.5,
#         'text-justify': 'auto',
#     }
# }

# # Add the names layer to the map
# m.add_layer(names_layer)

# Add layer control
m.add_layer_control()

# Display the map
m

Failed to retrieve the MapTiler style. Defaulting to OpenFreeMap 'liberty' style.


Map(height='600px', map_options={'bearing': 0, 'center': (-100, 40), 'pitch': 60, 'style': 'https://tiles.open…